In [1]:
import time

import requests
import json
from datetime import datetime
import re
from user_info import STR_COOKIE_MAIN, STR_COOKIE_EVERY_NOTE, URL_GET_NOTES, URL_GET_NOTE


# 排序指标，可以修改
def sortRule(obj):
    return obj['create_date']


# 将cookie字符串转为字典格式
def get_cookie(cookie_str):
    # 将cookie字符串转换为字典
    cookie_dict = {}
    for item in cookie_str.split('; '):
        key, value = item.split('=', 1)
        cookie_dict[key] = value
    return cookie_dict


def get_now():
    # 获取当前时间的时间戳（秒级别）
    timestamp_seconds = time.time()
    # 转换为毫秒级别
    timestamp_milliseconds = int(timestamp_seconds * 1000)
    return str(timestamp_milliseconds)

def get_folders():
    if len(folders_info) == 0:
        response = requests.get(URL_GET_NOTES, cookies=cookie_main)
        syncTag = response.json()['data']['syncTag']
        response = requests.get(URL_GET_NOTES+"&syncTag="+syncTag, cookies=cookie_main)
        for item in response.json()['data']['folders']:
            folders_obj[item['id']] = []
            folders_info[item['id']] = {"subject": item['subject'], "id": item['id'],
                                               "create_date": datetime.fromtimestamp(item['createDate'] / 1000),
                                               "modify_date": datetime.fromtimestamp(item['modifyDate'] / 1000)}


def nextPage(syncTag=None):
    url = URL_GET_NOTES
    ts = get_now()
    get_folders()   # 添加分类信息
    if syncTag:
        url += '&syncTag=' + syncTag

    url += '&ts=' + ts  # 添加时间戳参数
    try:
        response = requests.get(url, cookies=cookie_main)
    except:
        print("connection error")


    print("1: ")
    if response.status_code == 200:
        result = response.json()
        for entry in result['data']['entries']:
            print("2: ")
            detailUrl = URL_GET_NOTE + entry['id']
            try:
                detailResponse = requests.get(detailUrl, cookies=cookie_dict_every_note)
            except:
                print(f"error: {detailUrl} ")
                error_urls.append(detailUrl)
                continue

            if detailResponse.status_code == 200:
                detailInfo = detailResponse.json()['data']['entry']
                date = datetime.fromtimestamp(entry['createDate'] / 1000)
                modifyDate = datetime.fromtimestamp(entry['modifyDate'] / 1000)
                resultObj = {}
                resultObj['title'] = json.loads(detailInfo.get('extraInfo', '{}')).get('title', '无')
                resultObj['create_date'] = date
                resultObj['modify_date'] = modifyDate
                resultObj['content'] = detailInfo['content']
                resultObj['folderId'] = detailInfo['folderId']
                resultObj['colorId'] = detailInfo['colorId']
                print(detailUrl, '内容：', detailInfo['content'][:20])
                resultArray.append(resultObj)
                # 将笔记保存到分类字典中
                if (resultObj['folderId'] in folders_obj):
                    folders_obj[resultObj['folderId']].append(resultObj)
        if result['data']['entries']:
            nextPage(result['data']['syncTag'])


#             resultArray.sort(key=sortRule)
#             with open('output.txt', 'w', encoding='utf-8') as f:
#                 for resObj in resultArray:
#                     f.write('创建日期：' + dateFormat(resObj['create_date']) + '\n')
#                     f.write('修改日期：' + dateFormat(resObj['modify_date']) + '\n')
#                     f.write('标题：' + resObj['title'] + '\n')
#                     f.write('内容：' + remove_tags(resObj['content']) + '\n')
#                     f.write('-----------------------\n')
#                 print("内容已保存到 output.txt 文件中")

def dateFormat(date):
    return date.strftime('%Y-%m-%d %H:%M:%S')


def remove_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


def write_note(resObj: dict, f):
    f.write('创建日期：' + dateFormat(resObj['create_date']) + '\n')
    f.write('修改日期：' + dateFormat(resObj['modify_date']) + '\n')
    f.write('标题：' + resObj['title'] + '\n')
    f.write('内容：' + remove_tags(resObj['content']) + '\n')
    f.write('-----------------------\n')


def write_folder_info(folderObj: dict, f):
    f.write('创建日期：' + dateFormat(folderObj['create_date']) + '\n')
    f.write('修改日期：' + dateFormat(folderObj['modify_date']) + '\n')
    f.write('主题：' + folderObj['subject'] + '\n')
    f.write('----------------------------------------------\n')


if __name__ == "__main__":
    resultArray = []
    error_urls = []
    folders_info = dict()
    folders_obj = dict()
    # 将获取目录的cookie字符串复制到此
    str_cookie_main = STR_COOKIE_MAIN
    # 将获取单条笔记的cookie字符串复制到此
    str_cookie_every_note = STR_COOKIE_EVERY_NOTE
    # 将cookie字符串转为字典
    cookie_main = get_cookie(str_cookie_main)
    cookie_dict_every_note = get_cookie(str_cookie_every_note)
    nextPage()
    # 将笔记写入output.md文件中
    with open('output.md', 'w', encoding='utf-8') as f:
        for resObj in resultArray:
            write_note(resObj=resObj, f=f)
        print("内容已保存到 output.md 文件中")

    for folderObj in folders_info.values():
        folder_subject = folderObj['subject']
        folder_id = folderObj['id']
        with open(f"folders_{folder_subject}.md", 'w', encoding='utf-8') as f:
            write_folder_info(folderObj=folderObj, f=f)
            for _, resObj in folders_obj[folder_id]:
                write_note(resObj=resObj, f=f)
        print(f"内容已保存到 folders_{folder_subject}.md 文件中")

    # 将未成功的笔记链接保存到error_urls.txt文件
    with open('error_urls.txt', 'w', encoding='utf-8') as f:
        for url in error_urls:
            f.write(url + '\n')
        print("error 内容已保存到文件error_urls.txt中")


1: 
2: 
https://i.mi.com/note/note/17743945548054784 内容： 13833582083
2: 
https://i.mi.com/note/note/17743945548710176 内容： “一帆风顺的人会觉得航海也不过如此，却不
2: 
https://i.mi.com/note/note/17790601882779840 内容： “东南之美，有会稽之竹箭；西南之美，有华
2: 
https://i.mi.com/note/note/17910928758817184 内容： 大王卡在小米盒子里
2: 
https://i.mi.com/note/note/18029688789811488 内容： 电影《死亡诗社》的话：


　　 我们读
2: 
https://i.mi.com/note/note/17933537395491232 内容： 高考完一个多月了，现在想起临高考的日子，
2: 
https://i.mi.com/note/note/17978966471491648 内容： 初二   王畅  已缴费
初一   牛祎
2: 
https://i.mi.com/note/note/18221265020666272 内容： 她的q3440454002
2: 
https://i.mi.com/note/note/18553729197949344 内容： 110
2: 
https://i.mi.com/note/note/18823350004236448 内容： 周一周四晚上
六教南408 404  

2: 
https://i.mi.com/note/note/18991947137040544 内容： vidar招新


校级宣讲会时间10月
2: 
https://i.mi.com/note/note/19024074180673920 内容： 发短信分配年级:
董振阳 大一
牛邵龙 
2: 
https://i.mi.com/note/note/19027368437104672 内容： 20号中午，确定参评， 
<sound 
2: 
https://i.mi.com/note/note/19027668067828032 内容： 今天中午发短信要59份表和一张班级集体照
2: 


https://i.mi.com/note/note/27567527097680288 内容： 月亮南边，青藤在一次爬上窗台

我不再想
2: 
https://i.mi.com/note/note/27685864944582784 内容： 134
459457299

做一个封面
2: 
https://i.mi.com/note/note/27893758186766560 内容： 犹记去年七月一，沪上人民治垃圾
官民同心
2: 
https://i.mi.com/note/note/28116636700393504 内容： 期末
通信原理 考9道大题 pcm,基带
2: 
https://i.mi.com/note/note/28308593865933056 内容： ☺ 1485538246.m6bDVeM
2: 
https://i.mi.com/note/note/28309169788829792 内容： 1.实践对社会的作用是什么?
①实践是人
2: 
https://i.mi.com/note/note/28616044123275648 内容： 从我拿起笔，准备开始叙述你的细节开始，就
2: 
https://i.mi.com/note/note/28345854788387104 内容： 浪漫手机 周杰伦
半兽人 周杰伦
龙拳 
2: 
https://i.mi.com/note/note/28658100055851264 内容： “用兵之名有四，两相攻曰攻，以大加小曰伐
2: 
https://i.mi.com/note/note/28800503991124672 内容： 牙刷 充电器 耳机 眼药水 
2: 
https://i.mi.com/note/note/28829419007394240 内容： 
我是来自浙江衢州的詹康弘
那我就羽毛球
2: 
https://i.mi.com/note/note/28842263007216096 内容： 心理拓展:坐前三排有加分，总共上三次课，
2: 
https://i.mi.com/note/note/28908949085635328 内容： 首先美国孩子画的画不被接受传统水墨画艺术
2: 
https://i.mi.com/note/note/28946606162985024

https://i.mi.com/note/note/34654660110861600 内容： 7.28一个小时
7.29一个小时
7.
2: 
https://i.mi.com/note/note/35052448780272000 内容： <input type="checkbo
2: 
https://i.mi.com/note/note/35239795005342176 内容： 救一个人，是为世界留住了一个人，还是为一
2: 
https://i.mi.com/note/note/35651493229249536 内容： “不妨先做出点成绩，然后再去强调你的感受
2: 
https://i.mi.com/note/note/35651497376366784 内容： 如果神还不帮你，说明他相信你。
2: 
https://i.mi.com/note/note/35651556025451744 内容： 
☺ 1485538246.rYX-bn
2: 
https://i.mi.com/note/note/35688623886843936 内容： 回忆带给人的痛，往往是在思索中，感慨着时
2: 
https://i.mi.com/note/note/35762301801349120 内容： 我想很多人没有意识到自己的生命有多么宝贵
2: 
https://i.mi.com/note/note/35947077143380096 内容： 
☺ 1485538246.FMA6Ll
2: 
https://i.mi.com/note/note/32198932634812672 内容： 听说在岸上，就会忘了水下的事
什么时候到
2: 
https://i.mi.com/note/note/35948065563362560 内容： 爱是有勇气霸占一个人
2: 
https://i.mi.com/note/note/36031346262295392 内容： fate微众银行
2: 
https://i.mi.com/note/note/36038199989912480 内容： 实验结果图用python，画出来很好看，
2: 
https://i.mi.com/note/note/36046529160036544 内容： 错过一个不知道的事，就不会有错过

ValueError: too many values to unpack (expected 2)

In [2]:
for folderObj in folders_info.values():
    folder_subject = folderObj['subject']
    folder_id = folderObj['id']
    with open(f"folders_{folder_subject}.md", 'w', encoding='utf-8') as f:
        write_folder_info(folderObj=folderObj, f=f)
        for resObj in folders_obj[folder_id].values():
            write_note(resObj=resObj, f=f)
    print(f"内容已保存到 folders_{folder_subject}.md 文件中")

TypeError: string indices must be integers

In [2]:
folders_obj

{'36038197861171776': [{'title': '绘制论文插图',
   'create_date': datetime.datetime(2022, 11, 30, 0, 5, 42, 426000),
   'modify_date': datetime.datetime(2022, 11, 30, 0, 5, 48, 989000),
   'content': '实验结果图用python，画出来很好看，网上有各种教程，这个比较简单。\n架构图：process on &gt; viso = drawio(mac)',
   'folderId': '36038197861171776',
   'colorId': 0}],
 '36046534477234944': [{'title': '',
   'create_date': datetime.datetime(2022, 11, 23, 15, 4, 0, 480000),
   'modify_date': datetime.datetime(2022, 11, 23, 15, 6, 19, 922000),
   'content': '睡午觉，梦见在学生活动中心偶然碰到张潇和谭雯。他们也正好回头看见了我，为了避免产生我是故意偷看的误会，我转头就走了，返回宿舍区，却一直在回头看她们是不是在身后',
   'folderId': '36046534477234944',
   'colorId': 0},
  {'title': '',
   'create_date': datetime.datetime(2022, 11, 30, 11, 5, 21, 436000),
   'modify_date': datetime.datetime(2022, 11, 30, 11, 6, 54, 987000),
   'content': '今天梦见你和陈嘉豪在一起聊天，我误会了，醒来想起你确实是和别人在聊天',
   'folderId': '36046534477234944',
   'colorId': 0},
  {'title': '',
   'create_date': datetime.datetime(2022, 11, 16, 9, 59, 0, 869000